In [73]:
import os
import json
import requests
import datetime
import pandas as pd

In [74]:
def pv_data(startdate, enddate):
    url = f"https://api.solar.sheffield.ac.uk/pvlive/api/v4/pes/12?start={startdate}&end={enddate}&data_format=csv"
    responsepv = requests.get(url)
    data = responsepv.text.split('\n')[1:-1] #remove the last empty line
    #df = pd.DataFrame([x.split(',')for x in data], columns=['pes_id','time','output'],)
    #df['time'] = pd.to_datetime(df['time'], utc=True)
    #I freeze it beacuse you cannot add an df in dictionary
    return data

In [75]:
folder_path = "/Users/sukhdevchaman/Desktop/unitn/Big Data Techn/Projectenvbigdata/Project/new_forecasts_for_model"

In [76]:
data_dict = {}

In [77]:
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]

In [98]:
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]

for json_file in json_files:
    # Extract date and time from file name
    file_name = os.path.splitext(json_file)[0]
    date_time_str = file_name.split("_")[0] # Assumendo che il nome del file abbia il formato "data_ora.json"
    date_time = datetime.datetime.strptime(date_time_str, "%Y-%m-%d")
    
    # read the json file
    with open(os.path.join(folder_path, json_file), "r") as file:
        json_data = json.load(file)
    
    # Get the desired weather forecast from JSON file
    time = json_data["time"]
    temperature = json_data["temperature_2m"]
    humidity = json_data["relativehumidity_2m"]
    pressure = json_data["surface_pressure"]
    irradiance = json_data["direct_normal_irradiance"]
    windspeed = json_data["windspeed_10m"]
    winddirection = json_data["winddirection_10m"]
    
    # Run the REST API call to get the output of the photovoltaic panel
    startdate = date_time.strftime("Y-%m-%d")
    enddate = (date_time + datetime.timedelta(days=2)).strftime("Y-%m-%d")
    data = pv_data(startdate, enddate)

    # add the forecasts data in dict.
    data_dict[file_name] = {
        "time": time,
        "temperature": temperature,
        "humidity": humidity,
        "pressure": pressure,
        "irradiance": irradiance,
        "windspeed": windspeed,
        "winddirection": winddirection,
        "solar_output": data
    }


In [99]:
output_file_path = "/Users/sukhdevchaman/Desktop/unitn/Big Data Techn/Projectenvbigdata/Project/fullfile.json"
with open(output_file_path, "w") as output_file:
    json.dump(data_dict, output_file)